# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df=pd.read_csv("..\WeatherPy\output_data\weather.csv")
city_df


,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Tubualá,9.5167,-79.0333,78.33,86,100,9.24,PA,1627218500
1,Thinadhoo,0.5333,72.9333,81.73,72,44,6.17,MV,1627218501
2,Bogandé,12.9714,-0.1436,92.82,41,77,9.31,BF,1627218501
3,Saint George,37.1041,-113.5841,76.35,73,1,1.01,US,1627218502
4,Mogadishu,2.0371,45.3438,77.81,74,17,25.30,SO,1627218503
...,...,...,...,...,...,...,...,...,...
587,Ürümqi,43.8010,87.6005,92.35,17,20,4.47,CN,1627219125
588,Encruzilhada do Sul,-30.5439,-52.5219,65.73,58,20,6.71,BR,1627219125
589,Halifax,44.6453,-63.5724,75.52,79,0,1.99,CA,1627219056
590,Valparaíso,-33.0393,-71.6273,52.72,88,20,3.44,CL,1627219127


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_df["Humidity"]

In [5]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
smaller_df = city_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Clouds"]== 0) & \
                  (city_df["Max Temp"] >= 70) & (city_df["Max Temp"] <= 80)].dropna()
smaller_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
15,Camacupa,-12.0167,17.4833,79.63,21,0,6.76,AO,1627218517
19,Ilhabela,-23.7781,-45.3581,74.93,39,0,1.41,BR,1627218522
50,Goderich,43.7501,-81.7165,71.20,89,0,4.65,CA,1627218552
96,Caluquembe,-13.7833,14.6833,78.26,15,0,6.71,AO,1627218602
137,Presidencia Roque Sáenz Peña,-26.7852,-60.4388,78.06,42,0,7.40,AR,1627218651
200,Miracema,-21.4122,-42.1967,71.83,59,0,0.92,BR,1627218723
258,Cap Malheureux,-19.9842,57.6142,73.47,64,0,10.00,MU,1627218781
259,Brownwood,31.7093,-98.9912,77.00,74,0,7.07,US,1627218783
268,Guia Lopes da Laguna,-21.4578,-56.1142,79.11,32,0,8.39,BR,1627218793
280,Mossamedes,-15.1961,12.1522,78.06,43,0,9.42,AO,1627218805


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = smaller_df.loc[:,["City","Country","Lat","Lng"]]

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
15,Camacupa,AO,-12.0167,17.4833,
19,Ilhabela,BR,-23.7781,-45.3581,
50,Goderich,CA,43.7501,-81.7165,
96,Caluquembe,AO,-13.7833,14.6833,
137,Presidencia Roque Sáenz Peña,AR,-26.7852,-60.4388,
200,Miracema,BR,-21.4122,-42.1967,
258,Cap Malheureux,MU,-19.9842,57.6142,
259,Brownwood,US,31.7093,-98.9912,
268,Guia Lopes da Laguna,BR,-21.4578,-56.1142,
280,Mossamedes,AO,-15.1961,12.1522,


In [15]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
